In [4]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


In [16]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
train_size  = 4096
valid_size  = 256
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epochs = 200

In [13]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [14]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [17]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[:valid_size]
y_valid = y_train_all[:valid_size]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [18]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)) ,x_test.reshape((-1, *image_shape))

In [29]:
model = tf.keras.applications.ResNet50(include_top=True, weights=None, input_shape=(32, 32, 3), classes=class_num)

In [30]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          epochs=120,
          callbacks=None,
          validation_split=0.1)

Epoch 1/120
15/15 [==============================] - 22s 1s/step - loss: 2.2907 - accuracy: 0.1565 - val_loss: 2.3052 - val_accuracy: 0.0902
Epoch 2/120
15/15 [==============================] - 1s 62ms/step - loss: 2.2790 - accuracy: 0.1793 - val_loss: 2.3213 - val_accuracy: 0.0902
Epoch 3/120
15/15 [==============================] - 29s 2s/step - loss: 2.2514 - accuracy: 0.2073 - val_loss: 2.3705 - val_accuracy: 0.0902
Epoch 4/120
15/15 [==============================] - 1s 67ms/step - loss: 2.2359 - accuracy: 0.2225 - val_loss: 2.3706 - val_accuracy: 0.0902
Epoch 5/120
15/15 [==============================] - 1s 63ms/step - loss: 2.2238 - accuracy: 0.2349 - val_loss: 2.3709 - val_accuracy: 0.0902
Epoch 6/120
15/15 [==============================] - 1s 65ms/step - loss: 2.2165 - accuracy: 0.2434 - val_loss: 2.3707 - val_accuracy: 0.0902
Epoch 7/120
15/15 [==============================] - 1s 63ms/step - loss: 2.2111 - accuracy: 0.2463 - val_loss: 2.3704 - val_accuracy: 0.0902
Epoch 8/

15/15 [==============================] - 1s 64ms/step - loss: 2.2518 - accuracy: 0.2092 - val_loss: 2.3184 - val_accuracy: 0.1415
Epoch 59/120
15/15 [==============================] - 1s 63ms/step - loss: 2.2727 - accuracy: 0.1872 - val_loss: 2.3173 - val_accuracy: 0.1439
Epoch 60/120
15/15 [==============================] - 1s 65ms/step - loss: 2.2652 - accuracy: 0.1953 - val_loss: 2.3207 - val_accuracy: 0.1390
Epoch 61/120
15/15 [==============================] - 1s 63ms/step - loss: 2.2595 - accuracy: 0.2008 - val_loss: 2.3024 - val_accuracy: 0.1585
Epoch 62/120
15/15 [==============================] - 1s 64ms/step - loss: 2.2573 - accuracy: 0.2032 - val_loss: 2.2877 - val_accuracy: 0.1732
Epoch 63/120
15/15 [==============================] - 1s 64ms/step - loss: 2.2489 - accuracy: 0.2105 - val_loss: 2.2954 - val_accuracy: 0.1659
Epoch 64/120
15/15 [==============================] - 1s 64ms/step - loss: 2.2345 - accuracy: 0.2249 - val_loss: 2.2949 - val_accuracy: 0.1634
Epoch 65/120

In [25]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128])

==========NTK============
4/4 [==============================] - 1s 358ms/step - loss: 2.1563 - accuracy: 0.3047


[2.1563456058502197, 0.3046875]

In [26]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test)

==========MSE============
64/64 [==============================] - 1s 9ms/step - loss: 2.1528 - accuracy: 0.3018


[2.1527912616729736, 0.3017578125]

In [27]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128])

4/4 [==============================] - 0s 6ms/step - loss: 2.1674 - accuracy: 0.2578


[2.1674437522888184, 0.2578125]